In [15]:
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import numpy as np

from numpy import genfromtxt
from sklearn.datasets import load_boston
from tensorflow.python.saved_model.builder import SavedModelBuilder
from tensorflow.python.saved_model.signature_def_utils import build_signature_def
from tensorflow.python.saved_model.signature_constants import REGRESS_METHOD_NAME
from tensorflow.python.saved_model.tag_constants import TRAINING, SERVING
from tensorflow.python.saved_model.utils import build_tensor_info

In [16]:
# def read_dataset(filePath,delimiter=','):
#     return genfromtxt(filePath, delimiter=delimiter)

# def read_boston_data():
#     boston = load_boston()
#     features = np.array(boston.data)
#     labels = np.array(boston.target)
#     return features, labels

# def feature_normalize(dataset):
#     mu = np.mean(dataset,axis=0)
#     sigma = np.std(dataset,axis=0)
#     return (dataset - mu)/sigma

# def append_bias_reshape(features,labels):
#     n_training_samples = features.shape[0]
#     n_dim = features.shape[1]
#     f = np.reshape(np.c_[np.ones(n_training_samples),features],[n_training_samples,n_dim + 1])
#     l = np.reshape(labels,[n_training_samples,1])
#     return f, l

In [17]:
# features,labels = read_boston_data()
# normalized_features = feature_normalize(features)
# f, l = append_bias_reshape(normalized_features,labels)
# f = features
# n_dim = f.shape[1]

# rnd_indices = np.random.rand(len(f)) < 0.80

# train_x = f[rnd_indices]
# train_y = l[rnd_indices]
# test_x = f[~rnd_indices]
# test_y = l[~rnd_indices]

In [18]:
# train_x.shape, train_y.shape

In [19]:
learning_rate = 0.01
# training_epochs = 1000
cost_history = np.empty(shape=[1],dtype=float)

X = tf.placeholder(tf.float64,[None,n_dim], name="x")
X_test = tf.placeholder(tf.float64,[None,n_dim], name="x_test")
Y = tf.placeholder(tf.float64,[None,1], name="y")
W = tf.Variable(tf.ones([n_dim,1],dtype=tf.float64), name="w")

init = tf.variables_initializer(tf.global_variables(), name="init")

In [20]:
y_ = tf.matmul(X, W, name="y_hat")
y_preds = tf.matmul(X_test, W, name="y_preds")
cost = tf.reduce_mean(tf.square(y_ - Y))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost, name="train")

In [21]:
# X.shape

In [22]:
# sess = tf.Session()
# sess.run(init)

# for epoch in range(training_epochs):
#     sess.run(training_step,feed_dict={X:train_x,Y:train_y})
#     cost_history = np.append(cost_history,sess.run(cost,feed_dict={X: train_x,Y: train_y}))

In [23]:
# train_x.shape

In [24]:
# train_y.shape

In [25]:
# plt.plot(range(len(cost_history)),cost_history)
# plt.axis([0,training_epochs,0,np.max(cost_history)])
# plt.show()

In [26]:
# pred_y = sess.run(y_, feed_dict={X: test_x})
# mse = tf.reduce_mean(tf.square(pred_y - test_y))
# print("MSE: %.4f" % sess.run(mse)) 

# fig, ax = plt.subplots()
# ax.scatter(test_y, pred_y)
# ax.plot([test_y.min(), test_y.max()], [test_y.min(), test_y.max()], 'k--', lw=3)
# ax.set_xlabel('Measured')
# ax.set_ylabel('Predicted')
# plt.show()

http://aqibsaeed.github.io/2016-07-07-TensorflowLR/

In [27]:
# definition = tf.Session().graph_def
directory = 'boston_regression'
builder = SavedModelBuilder(directory)
# tf.train.write_graph(definition, directory, 'saved_model.pbtxt', as_text=True)

In [28]:
with tf.Session(graph=tf.get_default_graph()) as sess:
    sess.run(init)

    signature_inputs = {
        "x": build_tensor_info(X),
        "x_test": build_tensor_info(X_test),
        "y": build_tensor_info(Y)
    }
    signature_outputs = {
        "out": build_tensor_info(y_preds)
    }
    signature_def = build_signature_def(
        signature_inputs, signature_outputs,
        REGRESS_METHOD_NAME)
    builder.add_meta_graph_and_variables(
        sess, [TRAINING, SERVING],
        signature_def_map={
            REGRESS_METHOD_NAME: signature_def
        },
        assets_collection=tf.get_collection(tf.GraphKeys.ASSET_FILEPATHS))
    builder.save(as_text=False)

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: boston_regression/saved_model.pb
